In [1]:
# Adjustments
input_directory = 'raw_frames_gop_12_test'

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import shutil


# Initialize lists to store timestamps
timestamps = []

# Loop through the directory and extract timestamps from filenames
for filename in sorted(os.listdir(input_directory)):
    if filename.endswith('.raw'):
        # Extract the timestamp from the filename
        timestamp = int(filename.split('.')[0])
        timestamps.append(timestamp)

# Convert timestamps to numpy array
timestamps = np.array(timestamps)

# Subtract the first timestamp from all timestamps
timestamps_adjusted = timestamps - timestamps[0]

# Compute the differences between consecutive timestamps
time_differences = np.diff(timestamps_adjusted)

# Calculate the average time difference
average_time_difference = np.mean(time_differences)

# Create a constant value array (same length as timestamps)
constant_value = 1
values = np.full_like(timestamps_adjusted, constant_value)

# Calculate the expected frame interval for 24 FPS
fps = 24
frame_interval = 1000 / fps

# Generate expected timestamps
expected_timestamps = np.arange(0, timestamps_adjusted[-1] + frame_interval, frame_interval)
expected_values = np.full_like(expected_timestamps, constant_value)

# Count frames rendered in time
frames_rendered_in_time = 0
frames_rendered_late = 0
for index in range(len(timestamps_adjusted)):
    expected_time = expected_timestamps[index]
    actual_time = timestamps_adjusted[index]
    if actual_time <= expected_time:
        # print("In time", actual_time, expected_time)
        frames_rendered_in_time += 1
    else:
        # print("Late", actual_time, expected_time)
        frames_rendered_late += 1
print("Frames rendered in time:", frames_rendered_in_time)
print("Frames rendered late:", frames_rendered_late)


# Plot the actual and expected values
# plt.figure(figsize=(12, 8))
# plt.plot(timestamps_adjusted, values, 'o', label='Actual Values', color='blue', markersize=4)
# # plt.plot(expected_timestamps, expected_values, 'o', label='Expected 24 FPS Values', color='red', markersize=0.1)
# plt.xlabel('Render Timestamp (ms)')
# plt.ylabel('Value')
# plt.title('Actual vs Expected Render Timestamps')
# plt.legend()
# plt.grid(True)

# Print the average render time difference
print(f"Average render time difference: {average_time_difference:.2f} ms")

# plt.savefig(f'plots/actual-render-times-gop-48-idle.pdf', bbox_inches='tight')

Frames rendered in time: 77
Frames rendered late: 340
Average render time difference: 47.88 ms


In [3]:
# Configuration
output_directory = 'numbered_raw_frames'  # Path to store new .raw files
frame_rate = 24  # Frames per second
timestamp_interval = 1000 / frame_rate  # Interval in milliseconds
number_of_original_frames = 480

# Ensure output directory exists
os.makedirs(output_directory, exist_ok=True)

# List and sort original .raw files by timestamp
files = [f for f in os.listdir(input_directory) if f.endswith('.raw')]
files.sort(key=lambda f: int(f.split('.')[0]))

# Find the earliest timestamp
earliest_timestamp = int(files[0].split('.')[0])

# Normalize timestamps and rename files
normalized_files = []
for file in files:
    old_timestamp = int(file.split('.')[0])
    new_timestamp = old_timestamp - earliest_timestamp
    new_filename = f'{new_timestamp}.raw'
    normalized_files.append(new_filename)
    shutil.copy(os.path.join(input_directory, file), os.path.join(output_directory, new_filename))

print(f"Renamed and copied {len(normalized_files)} files.")

# Create a series of new files based on frame timestamps
current_time = 0
frame_number = 0

while frame_number < number_of_original_frames:
    # Find the latest file before or at the current timestamp
    closest_file = None
    for file in reversed(normalized_files):
        file_timestamp = int(file.split('.')[0])
        print(file_timestamp, current_time)
        if file_timestamp <= current_time:
            closest_file = file
            break
    
    if closest_file is None:
        break  # No more files to process

    # Construct output filename
    output_filename = f'frame-{frame_number}.raw'
    output_file_path = os.path.join(output_directory, output_filename)

    # Copy the closest file to the new series
    shutil.copy(os.path.join(output_directory, closest_file), output_file_path)
    
    # Move to the next timestamp
    current_time += timestamp_interval
    frame_number += 1

# Loop through the list and delete each file
for file_name in normalized_files:
    file_path = os.path.join(output_directory, file_name)
    
    try:
        if os.path.exists(file_path):
            os.remove(file_path)
            # print(f"Deleted: {file_path}")
        else:
            print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

print(f"Processed {frame_number} frames.")


Renamed and copied 417 files.
19919 0
19871 0
19842 0
19791 0
19756 0
19703 0
19673 0
19627 0
19587 0
19539 0
19504 0
19459 0
19420 0
19370 0
19342 0
19287 0
19252 0
19205 0
19172 0
19124 0
19087 0
19037 0
19002 0
18985 0
18920 0
18871 0
18835 0
18790 0
18749 0
18703 0
18675 0
18622 0
18580 0
18538 0
18509 0
18456 0
18411 0
18373 0
18339 0
18296 0
18248 0
18212 0
18182 0
18123 0
18083 0
18049 0
18025 0
17953 0
17913 0
17878 0
17839 0
17793 0
17751 0
17704 0
17674 0
17620 0
17582 0
17538 0
17505 0
17462 0
17413 0
17372 0
17344 0
17287 0
17246 0
17206 0
17171 0
17122 0
17085 0
17038 0
16995 0
16980 0
16918 0
16871 0
16835 0
16787 0
16748 0
16706 0
16675 0
16624 0
16585 0
16538 0
16505 0
16460 0
16415 0
16373 0
16336 0
16294 0
16246 0
16207 0
16169 0
16122 0
16081 0
16038 0
16006 0
15954 0
15915 0
15879 0
15840 0
15788 0
15744 0
15710 0
15666 0
15624 0
15583 0
15539 0
15501 0
15454 0
15416 0
15371 0
15336 0
15288 0
15247 0
15208 0
15169 0
15123 0
15080 0
15038 0
14998 0
14984 0
14913 0
14

In [4]:
### Received video frames just stitched together ###

# # Configuration
# output_directory = 'numbered_raw_frames'  # Path to store new .raw files
# frame_rate = 24  # Frames per second
# timestamp_interval = 1000 / frame_rate  # Interval in milliseconds
# number_of_original_frames = 240

# # Ensure output directory exists
# os.makedirs(output_directory, exist_ok=True)

# # List and sort original .raw files by timestamp
# files = [f for f in os.listdir(input_directory) if f.endswith('.raw')]
# files.sort(key=lambda f: int(f.split('.')[0]))

# # Normalize timestamps and rename files
# normalized_files = []
# for index, file in enumerate(files):
#     old_timestamp = int(file.split('.')[0])
#     new_filename = f'frame-{index}.raw'
#     normalized_files.append(new_filename)
#     shutil.copy(os.path.join(input_directory, file), os.path.join(output_directory, new_filename))

# print(f"Renamed and copied {len(normalized_files)} files.")